In [2]:
import os.path as osp
from tqdm.auto import tqdm
import numpy as np
import wandb

import torch
from sklearn.metrics import roc_auc_score

import torch.nn.functional as F
# from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
from torch.nn import LayerNorm, Linear, ReLU

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GatedGraphConv
from torch_geometric.nn import DeepGCNLayer, GENConv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import DataLoader

from dataset_processing import RNADataset


In [3]:
dataset = RNADataset(root="./data/")
dataset = dataset.shuffle()
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]
bs = 4

train_dataloader = DataLoader(train_data, batch_size=bs, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)


In [4]:
train_data[1].adj_mat

torch.Size([196, 196])

In [4]:
def del_nucl_conn(y):
#     main_diag = torch.zeros([1, y.size()[0] // bs])
#     diag1 = torch.ones([1, (y.size()[0] // bs) - 1])
    
    main_diag = torch.zeros([1, y.size()[0]])
    diag1 = torch.ones([1, y.size()[0] - 1])
#     batch_diags = []
#     for i in range(bs):
#         batch_diags.append((torch.diag_embed(main_diag).to(device) +
#             torch.diag_embed(diag1, offset=1).to(device) + 
#             torch.diag_embed(diag1, offset=-1).to(device) - 1)[0])

#     return y * torch.cat(batch_diags, 0).to(device) * (-1)
    return y * (torch.diag_embed(main_diag).to(device) + 
                    torch.diag_embed(diag1, offset=1).to(device) + 
                torch.diag_embed(diag1, offset=-1).to(device) - 1)[0] * (-1)


def del_main_diag(y):
#     main_diag = torch.ones([1, y.size()[0] // bs])
    main_diag = torch.zeros([1, y.size()[0]])

#     batch_diags = []
#     for i in range(bs):
#         batch_diags.append((torch.diag_embed(main_diag).to(device) - 1)[0])
    
#     return y * torch.cat(batch_diags, 0).to(device) * (-1)
    return y * (torch.diag_embed(main_diag).to(device) - 1)[0] * (-1)


def adj_mat_split(y):
    y = list(torch.split(y, 196, dim=0))
#     print(y)
    for i in range(len(y)):
        y[i] = torch.split(y[i], 196, dim=1)
    
    y1 = []
    for i in range(len(y)):
        y1.append(y[i][i])
    
    return torch.cat(y1, 0).to(device)
    

def argm(y):
    max_ind = torch.argmax(y, dim=1)
    y1 = torch.zeros_like(y)
    k = 0
    for i in max_ind:
        y1[k][i] = 1.
        y1[i][k] = 1.
        k += 1
    
    return y1 * y


In [5]:
a = torch.rand([4, 4])
print(a)
adj_mat_split(a)

tensor([[0.4004, 0.8519, 0.5511, 0.0188],
        [0.8301, 0.4703, 0.4650, 0.6933],
        [0.4348, 0.7514, 0.2455, 0.0012],
        [0.9674, 0.0052, 0.9604, 0.7480]])


NameError: name 'device' is not defined

In [6]:
def precision(y_pred, y_true):
    y_pred = del_nucl_conn(y_pred)
#     y_pred[(y_pred > 0.5)] = 1
#     y_pred[(y_pred <= 0.5)] = 0 
#     y_pred = argm(y_pred)
    y_pred = del_main_diag(y_pred)
    y_pred = adj_mat_split(y_pred)

    tp = torch.sum(y_pred * y_true)
    fp = torch.sum((1 - y_true) * y_pred)
    
    return tp / (tp + fp + epsilon)

def recall(y_pred, y_true):
    y_pred = del_nucl_conn(y_pred)
#     y_pred[(y_pred > 0.5)] = 1
#     y_pred[(y_pred <= 0.5)] = 0 
#     y_pred = argm(y_pred)
    y_pred = del_main_diag(y_pred)
    y_pred = adj_mat_split(y_pred)

    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    
    return tp / (tp + fn + epsilon)

def f_loss(y_pred, y_true):
    y_pred = del_nucl_conn(y_pred)
#     y_pred = argm(y_pred)
    y_pred = del_main_diag(y_pred)
    y_pred = adj_mat_split(y_pred)

    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    fp = torch.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    k = 1
    k1 = 1 - torch.abs(precision - recall)

#     k2 = 1 - torch.abs(K.mean(precision) - K.mean(recall))
    #calculate upgraded f1 score
    f1 = ((1 + k * k) * precision * recall) / (k * k * precision + recall + epsilon)
#     tw = K.sum(K.cast(y_true * y_pred, ’float32’), axis=[1, 2, 3])
#     fw = K.sum(K.cast((1 - y_true) * y_pred, ’float32’), axis=[1, 2, 3])
#     fb = K.sum(K.cast(y_true * (1 - y_pred), ’float32’), axis=[1, 2, 3])
    return 1 - f1


In [28]:
class DeepGCNModel(torch.nn.Module):
    def __init__(self, num_layers, out_channels):
        super(DeepGCNModel, self).__init__()
        self.node_encoder = Linear(dataset.num_features, out_channels)
#         self.edge_encoder = Linear(train_data.edge_attr.size(-1), hidden_channels)

        self.layers = torch.nn.ModuleList()
        for i in range(1, num_layers + 1):
#             conv = GENConv(hidden_channels, hidden_channels, aggr='softmax',
#                            t=1.0, learn_t=True, num_layers=2, norm='layer')
            conv = GatedGraphConv(out_channels, num_layers=4)
            norm = LayerNorm(out_channels, elementwise_affine=False)
            act = ReLU()

            layer = DeepGCNLayer(conv, block='res')
            self.layers.append(layer)

        self.lin = Linear(out_channels, 1)

    def forward(self, x, edge_index):
#         x = self.node_encoder(x)
#         edge_attr = self.edge_encoder(edge_attr)
#         print(x)
        x = self.layers[0].conv(x, edge_index)
#         print(x)
        for layer in self.layers[1:]:
            x = layer(x, edge_index)
#         x = self.layers[0].act(self.layers[0].norm(x))
#         x = F.dropout(x, p=0.1, training=self.training)
#         print(x)
        s = x.size()[0]
#         for i in range(s):
#             print(self.lin(x[i]))
#             x[i] = self.lin(x[i])
        
#         return self.lin(x)
        return x.sigmoid()

In [29]:
num_layers = 6
# num_features = dataset.num_features
out_channels = 200


lr = 0.00001
epochs = 1500
beta = 1


model = DeepGCNModel(num_layers, out_channels)
# model = torch.load("./models1/GatedGCN_6_4e-05_Adam_200_2_60.pt")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_data = list(map(lambda x: x.to(device), train_data))
val_data = list(map(lambda x: x.to(device), val_data))
print(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
# optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=lr)

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# criterion = torch.nn.BCELoss()
# criterion = torch.nn.BCEWithLogitsLoss()
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.KLDivLoss()

# def RMSELoss(y_pred, y_true):
#     return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

criterion = f_loss

cuda


In [31]:
model(train_data[0].x, train_data[0].edge_index)

tensor([[-0.0606],
        [-0.1172],
        [-0.1171],
        [-0.1886],
        [-0.1391],
        [-0.1822],
        [-0.1245],
        [-0.1004],
        [-0.0970],
        [-0.1148],
        [-0.1130],
        [-0.1252],
        [-0.1241],
        [-0.0981],
        [-0.1179],
        [-0.0809],
        [-0.1006],
        [-0.1143],
        [-0.1180],
        [-0.1343],
        [-0.1245],
        [-0.1651],
        [-0.0910],
        [-0.0969],
        [-0.0785],
        [-0.0734],
        [-0.0640],
        [-0.0574],
        [-0.0708],
        [-0.0807],
        [-0.0749],
        [-0.0888],
        [-0.0859],
        [-0.1144],
        [-0.0947],
        [-0.1097],
        [-0.0997],
        [-0.1064],
        [-0.1456],
        [-0.1173],
        [-0.1662],
        [-0.1096],
        [-0.0804]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [36]:
run_name = "ResGCN_bs4_" + str(num_layers) + "*4_" + str(lr) + "_" + "Adam_" + str(out_channels) + "_" + str(beta)


wandb.init(
    # set the wandb project where this run will be logged
    project="secondary_structure_prediction1",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "DeepGCN",
    "epochs": epochs,
    "optimizer": "Adam",
    "out_channels": out_channels,
    "loss": "f_loss",
    "beta": beta,
    "train:val:test": "655:218:218"
    },
    name=run_name
)

epsilon = 1e-10

train_f1,▁▅▆▆▆▆▆▆▇██████▆▇
train_loss,█▄▃▃▃▃▃▃▂▁▁▁▁▁▁▃▂
train_precision,▁▅▆▆▆▆▆▆▇██████▆▇
train_recall,▃▅▄▅▄▄▃▃▄██████▁▄
val_f1,▁▅▆▆▆▆▆█▇██████▇█
val_loss,█▄▃▃▃▃▃▁▂▁▁▁▁▁▁▂▁
val_precision,▁▅▆▆▆▆▆█▇██████▇█
val_recall,▁▆▇▅▅▃▂▇▅█████▇▅█
train_f1,0.00369
train_loss,0.99631
train_precision,0.00185


In [37]:
def train():
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        train_recall = []
        train_precision = []
        for g in tqdm(train_dataloader):
            g.to(device)
            optimizer.zero_grad()

            out = model(g.x, g.edge_index)
            y_true = g.adj_mat
#             print(out.size(),y_true.size())
            loss = criterion(out, y_true.to(torch.float32))            
            
            loss.backward()
            optimizer.step()
            
#             out = out.sigmoid()
            
            train_loss.append(loss.item())
            train_precision.append(precision(out, y_true).item())
            train_recall.append(recall(out, y_true).item())
            
        train_prec = np.mean(train_precision)
        train_rec = np.mean(train_recall)
        train_f1 = (2 * train_prec * train_rec) / (train_prec + train_rec)
        print(f'Epoch: {epoch:03d}, loss: {np.mean(train_loss)}, f1: {train_f1}, precision: {train_prec}, recall: {train_rec}')
        
        val_loss = []
        val_recall = []
        val_precision = []
        with torch.no_grad():
            for g in val_dataloader:
                g.to(device)
                out = model(g.x, g.edge_index)
        
                y_true = g.adj_mat
                loss = criterion(out, y_true.to(torch.float32))
                                
                val_loss.append(loss.item())
                
#                 out = out.sigmoid()
                
                val_precision.append(precision(out, y_true).item())
                val_recall.append(recall(out, y_true).item())
 
            
            prec = np.mean(val_precision)
            rec = np.mean(val_recall)
            f1 = (2 * prec * rec) / (prec + rec)
            print(f'val_loss: {np.mean(val_loss)}, val_f1: {f1}, val_precision: {prec}, val_recall: {rec}')
              
            wandb.log({"train_loss": np.mean(train_loss), "train_f1": train_f1, "train_precision": train_prec, 
                       "train_recall": train_rec,
                       "val_loss": np.mean(val_loss), "val_f1": f1, "val_precision": prec, "val_recall": rec})
#     wandb.finish()

In [38]:
train()
torch.save(model, "./models1/" + run_name + "_" + str(epochs) + ".pt")

  0%|          | 0/164 [00:00<?, ?it/s]

Epoch: 001, loss: 0.9970172232970959, f1: 0.0029833825243262557, precision: 0.0014939595653786233, recall: 0.9824640790863735
val_loss: 0.9962828766315355, val_f1: 0.003720608049378942, val_precision: 0.0018638778751456486, val_recall: 0.9702083383678296


  0%|          | 0/164 [00:00<?, ?it/s]

Epoch: 002, loss: 0.9968186758640336, f1: 0.003182029413234458, precision: 0.0015935917907016828, recall: 0.9838359526744703
val_loss: 0.9961646035176899, val_f1: 0.0038388646519574845, val_precision: 0.0019232413504291924, val_recall: 0.9691540883221758


  0%|          | 0/164 [00:00<?, ?it/s]

...

  0%|          | 0/164 [00:00<?, ?it/s]

Epoch: 045, loss: 0.9947294291199708, f1: 0.005271417465046268, precision: 0.002642729982631508, recall: 0.9920550308576445
val_loss: 0.9945021318186313, val_f1: 0.005501672988918722, val_precision: 0.0027585388807400447, val_recall: 0.9851868171757514


  0%|          | 0/164 [00:00<?, ?it/s]

Epoch: 046, loss: 0.9947260843544472, f1: 0.005274764437320921, precision: 0.002644412399471238, recall: 0.9920550308576445


KeyboardInterrupt: 